In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 

import models.logistic_regression.logistic_regression as lr
from samplers import sgd as sgd
from samplers import sgld as sgld
from samplers import sgldps as sgldps
from samplers import sgldcv as sgldcv
from samplers import sgldcvps as sgldcvps

key = random.PRNGKey(11)

In [2]:
#set up model-specific gradient functions
gradf_0 = lr.gradf_0
gradf_i_batch = lr.gradf_i_batch
post_var = lr.post_var

### Set up model and data

In [3]:
#load in the data
file_path = "../../data/real/covtype_train.csv"
test_path = "../../data/real/covtype_test.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]
test_data = pd.read_csv(test_path)
test_array = test_data.values[:]
N_test = test_array.shape[0]
x_test = np.column_stack([np.ones(N_test), test_array[:, 1:]])
y_test = test_array[:,0]

#set up model parameters
dim = x.shape[1] 

#priors
mu_0 = np.zeros(dim) #prior mean
lambda_0 = 10.0*np.eye(dim)  #prior covariance matrix

### Set up sampling framework

In [4]:
# step-size
step_size = 10**(-6)
step_sgd = 1e-3
# batch sizes
n_batch = np.int64(N*0.001)
# number of chains (first chain discarded due to slower runtime)
N_rep = 11
#iterations
burnin = np.int64(10/0.001)
Niter = 2*burnin

### Running SGLD-CV (0.001N) sampler

In [5]:
runtime_df=[]
samples_df = []
grads_df = []

In [6]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n_batch, step_sgd, replacement=True)
    samples, grads, runtime = sgldcv.sgld_cv_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n_batch, replacement=True)
    
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime))) #join output
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)    

Number of chains run: 100%|██████████| 11/11 [09:00<00:00, 49.11s/it]


In [7]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df)) #save output
runtime_df.to_csv("./out/lrb_sgldcv_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldcv_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldcv_grads.csv", index=False)

### Running SGLD-CV-PS (0.001N) sampler

In [8]:
runtime_df=[]
samples_df = []
grads_df = []

In [9]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n_batch, step_sgd, replacement=True)
    samples, grads, runtime, probs = sgldcvps.sgldcv_ps_sampler(subkey, gradf_0, gradf_i_batch, post_var, burnin, step_size, theta_hat, theta_hat, x, y, n_batch)
    
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime)))
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [1:04:21<00:00, 351.01s/it]


In [10]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgldcvps_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldcvps_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldcvps_grads.csv", index=False)

### Running SGLD (0.001N) sampler

In [11]:
runtime_df=[]
samples_df = []
grads_df = []

In [12]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    samples, grads, runtime = sgld.sgld_sampler(subkey, gradf_0, gradf_i_batch, Niter, step_size, theta_0, x, y, n_batch, replacement=True)
    runtime_df.append(runtime)
    samples_df.append(samples)
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [10:07<00:00, 55.25s/it]


In [13]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgld_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgld_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgld_grads.csv", index=False)

### Running SGLD-PS (0.001N) sampler 

In [14]:
runtime_df=[]
samples_df = []
grads_df = []

In [15]:
for i in tqdm(range(N_rep), desc = "Number of chains run"):
    key, subkey = random.split(key)
    theta_0 = jnp.zeros(dim)
    theta_hat, samples_sgd, runtime_sgd = sgd.adam(key, gradf_0, gradf_i_batch, burnin, theta_0, x, y, n_batch, step_sgd, replacement=True)
    samples, grads, runtime = sgldps.sgldps_sampler(subkey, gradf_0, gradf_i_batch, burnin, step_size, theta_hat, theta_hat, x, y, n_batch,  prob_type='approx')
        
    runtime_df.append(np.concatenate((runtime_sgd, runtime_sgd[burnin-1]+runtime)))
    samples_df.append(np.concatenate((samples_sgd, samples[1:]), axis=0))
    grads_df.append(grads)

Number of chains run: 100%|██████████| 11/11 [1:00:30<00:00, 330.08s/it]


In [16]:
runtime_df = pd.DataFrame(np.column_stack(runtime_df))
runtime_df.to_csv("./out/lrb_sgldps_runtime.csv", index=False)
samples_df = pd.DataFrame(np.column_stack(samples_df))
samples_df.to_csv("./out/lrb_sgldps_samples.csv", index=False)
grads_df = pd.DataFrame(np.column_stack(grads_df))
grads_df.to_csv("./out/lrb_sgldps_grads.csv", index=False)